In [1]:
import os
import re
from collections import Counter

import anndata
import scanpy as sc
import numpy as np
import pandas as pd

import anndata2ri

In [2]:
# Activate the anndata2ri conversion between SingleCellExperiment and AnnData
anndata2ri.activate()

#Loading the rpy2 extension enables cell magic to be used
#This runs R code in jupyter notebook cells
%load_ext rpy2.ipython

In [3]:
# input/output filepaths
rds_fp = '/home/estorrs/data/single_cell_classification/tumor/BR/scRNA/brca_with_immune.rds'
output_fp = '/home/estorrs/data/single_cell_classification/tumor/BR/scRNA/brca_with_immune_v2.h5ad'

In [4]:
%%R -i rds_fp
suppressPackageStartupMessages(library(Seurat))

final = readRDS(file = rds_fp)

In [5]:
%%R -o metadata -o counts -o sctransform -o sctransform_genes -o genes -o umap
# add the active idents to the metadata
final = AddMetaData(final, unname(final@active.ident), col.name = 'active.ident')

# grab metadata
metadata = final@meta.data

# grab umap embeddings
umap = Embeddings(final[["umap"]])

# grab gene names
genes = rownames(GetAssayData(object = final, slot = "counts", assay = "RNA"))

# get raw counts
counts = final@assays$RNA@counts

# get sctransform genes and output
sctransform_genes = rownames(GetAssayData(object = final, slot = "data", assay = "SCT"))
sctransform = final@assays$SCT@data

In [6]:
counts.shape, sctransform.shape

((33538, 98564), (27131, 98564))

In [8]:
metadata

,orig.ident,nCount_RNA,nFeature_RNA,percent.mito,nCount_SCT,nFeature_SCT,SCT_snn_res.0.5,seurat_clusters,sample,tissue_type,cell_type,cell_type_specific,Piece_ID,Clinical_Subtype,doublet_score,predicted_doublet,Composite_PAM50,active.ident
_HT062B1_S1PA_AAACCCACACAAATGA-1,TWCE-HT062B1-S1PAA1A1Z1B1,4837.0,1378,0.043829,4154.0,1378,2,2,HT062B1,tumor,CD8_T,CD8_CTL,HT062B1_S1PA,TNBC,0.036018,0,Her2,CD8_CTL
_HT062B1_S1PA_AAACCCAGTGCTCCGA-1,TWCE-HT062B1-S1PAA1A1Z1B1,6400.0,2369,0.061562,4515.0,2344,4,4,HT062B1,tumor,Endothelial,Endothelial,HT062B1_S1PA,TNBC,0.105210,0,Her2,Endothelial
_HT062B1_S1PA_AAACCCATCGGAATTC-1,TWCE-HT062B1-S1PAA1A1Z1B1,10255.0,2945,0.042321,4558.0,1978,10,10,HT062B1,tumor,CAF,mCAF,HT062B1_S1PA,TNBC,0.051376,0,Her2,mCAF
_HT062B1_S1PA_AAACGAACAGCTAACT-1,TWCE-HT062B1-S1PAA1A1Z1B1,1433.0,751,0.073273,3072.0,801,13,13,HT062B1,tumor,Tumor,Tumor,HT062B1_S1PA,TNBC,0.013000,0,Her2,Tumor
_HT062B1_S1PA_AAACGAAGTAGGGAGG-1,TWCE-HT062B1-S1PAA1A1Z1B1,2256.0,934,0.050089,3281.0,934,2,2,HT062B1,tumor,CD8_T,CD8_T preexhausted GZMK+,HT062B1_S1PA,TNBC,0.027729,0,Her2,CD8_T preexhausted GZMK+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
_HT171B1_BC2_TTTGTTGCACCAGACC-1,TWCE-HT171B1-BC2,4717.0,1660,0.063600,4178.0,1660,8,8,HT171B1,tumor,CD4_T,CD4_T exhausted,HT171B1_S1H8,TNBC,0.104260,0,Basal,CD4_T exhausted
_HT171B1_BC2_TTTGTTGCACGTACTA-1,TWCE-HT171B1-BC2,4627.0,1441,0.014048,4117.0,1441,1,1,HT171B1,tumor,Tumor,Tumor,HT171B1_S1H8,TNBC,0.034114,0,Basal,Tumor
_HT171B1_BC2_TTTGTTGTCCCAAGCG-1,TWCE-HT171B1-BC2,3236.0,1392,0.051916,3482.0,1392,8,8,HT171B1,tumor,CD8_T,CD8_T exhausted,HT171B1_S1H8,TNBC,0.027347,0,Basal,CD8_T exhausted
_HT171B1_BC2_TTTGTTGTCCTCACCA-1,TWCE-HT171B1-BC2,13163.0,4124,0.036162,3946.0,1808,7,7,HT171B1,tumor,Mono_Macro,Mono_Macro,HT171B1_S1H8,TNBC,0.233279,0,Basal,Mono_Macro


In [9]:
# create new anndata object with sctransform normalized counts
adata = anndata.AnnData(X=counts.transpose(), obs=metadata,)
adata.var.index = genes

# filter to sctransform genes and put raw counts into different layer to make room for sctransform data
# comment out these 3 lines if you don't need the sctransform expression and are only interested in raw counts
adata = adata[:, sctransform_genes]
adata.layers['counts'] = adata.X.copy()
adata.X = sctransform.transpose()

# carry over umap and is_highly_variable from seurat
adata.obsm['X_umap'] = np.asarray(umap)

adata

AnnData object with n_obs × n_vars = 98564 × 27131
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mito', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.5', 'seurat_clusters', 'sample', 'tissue_type', 'cell_type', 'cell_type_specific', 'Piece_ID', 'Clinical_Subtype', 'doublet_score', 'predicted_doublet', 'Composite_PAM50', 'active.ident'
    obsm: 'X_umap'
    layers: 'counts'

In [10]:
# some versions of scanpy save the .h5ad weird so making sure columns are unicode strings before saving
adata.obs.columns = adata.obs.columns.astype(str)
adata.var.columns = adata.var.columns.astype(str)
adata.write_h5ad(output_fp)

... storing 'orig.ident' as categorical
... storing 'sample' as categorical
... storing 'tissue_type' as categorical
... storing 'Piece_ID' as categorical
... storing 'Clinical_Subtype' as categorical
... storing 'Composite_PAM50' as categorical
